In [6]:
import sys
import os


dirname = os.path.realpath("../src") #os.path.dirname(os.path.realpath("../AP/"))
print(dirname)
if dirname not in sys.path:
    sys.path.insert(0, dirname )
#sys.path.insert(0, "/Users/wenfengwang/go/src/github.com/mistsys/mist-topologies/resource/resources/mist-rrm-exp")
#print sys.path


import time
import boto3
from botocore.exceptions import ClientError

#import boto_utils

def run_query(query, database, s3_output):
    client = boto3.client('athena')
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': 's3://mist-{ENV}-athena/results/'.format(ENV=env)
        }
    )
    # print('Execution ID: ' + response['QueryExecutionId'])
    print(response)
    return response


/opt/mistsys/ds_incubator/src


In [7]:
!pip install boto3

In [8]:
env = 'staging'
env = "production"

database = 'secorapp-{ENV}'.format(ENV=env)
table = 'cv_ap_scans'
# table = 'rrm_apscan'

bucket = 'mist-{ENV}-athena'.format(ENV=env)
s3_ouput = 's3://%s/results/' % bucket
download_path = os.curdir + '/../downloads/'
print("download_path = {}".format(download_path))

download_path = ./../downloads/


In [9]:
site='a799e12a-fc28-4ce7-a563-c2dd6d498e2f '
site = 'a40ba08c-5e27-40d9-b798-7020ca6e4cae'

query = '''SELECT date_trunc('hour', time) as time, org, site, ap, band, channel, ap2,
 count(ap2) as counts, max(rssi) as max_rssi, max(in_site) as in_site
 FROM "secorapp-{ENV}"."{TABLE}"
 WHERE site="{SITE}" and time > date_add('day',-1,now())
 GROUP BY time,org,site,ap,band,ap2,channel;'''.format(ENV=env,TABLE=table,SITE=site)

query = '''SELECT * FROM "secorapp-{ENV}"."{TABLE}" 
where site='{SITE}' and band='{BAND}'.format(ENV=env,TABLE=table,SITE=site, BAND=band)
'''
print(query)

response = run_query(query, database, s3_ouput)

if response.get('ResponseMetadata').get('HTTPStatusCode') == 200:
    id = response.get('QueryExecutionId')
    file = 'results/' + id + '.csv'
    while True:
        client = boto3.client('s3')
        s3_output_response = ""
        try:
            s3_output_response = client.get_object(Bucket=bucket, Key=file)
            print(s3_output_response)
            client.download_file(Bucket=bucket, Key=file, Filename=download_path + id + '.csv')
            break
        except ClientError:
            print('checking if key exist')
        time.sleep(5)

SELECT * FROM "secorapp-{ENV}"."{TABLE}" 
where site='{SITE}' and band='{BAND}'.format(ENV=env,TABLE=table,SITE=site, BAND=band)



NoRegionError: You must specify a region.

In [6]:
!ls ../downloads/results/
#filename = download_path+file

f92186ad-642b-4ec7-9d3b-7cd4aa33c492.csv


In [7]:
import pandas as pd

filename = "../downloads/results/f92186ad-642b-4ec7-9d3b-7cd4aa33c492.csv"
df = pd.read_csv(filename)
df.head(2)

,ap,band,bandwidth,channel,insite_channel_weight,max_tx_power,noise_floor,num_clients,org,radar,...,site,timestamp,tx_power,util_all,util_nonwifi,util_rxinbss,util_rxotherbss,util_tx,partition_0,dt
0,5c5b352e113f,5,20,157,0.459861,11,-87,0,56de201d-e63b-4312-9858-40f4cfe35c7f,0,...,a40ba08c-5e27-40d9-b798-7020ca6e4cae,1970-01-18 20:09:18.158,NaN,0.168678,0.002074,0.000027,0.14389,0.015231,rrm-apscan-production,2018-11-04
1,5c5b352e113f,5,20,157,0.348852,11,-87,0,56de201d-e63b-4312-9858-40f4cfe35c7f,0,...,a40ba08c-5e27-40d9-b798-7020ca6e4cae,1970-01-18 20:09:18.158,NaN,0.168678,0.002074,0.000027,0.14389,0.015231,rrm-apscan-production,2018-11-04


In [9]:
import csv

with open(filename) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if line_count< 10:
                 print(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are ap, band, bandwidth, channel, insite_channel_weight, max_tx_power, noise_floor, num_clients, org, radar, scan_channel, scan_channel_weight, scan_congest_nonwifi, scan_congest_rxotherbss, scan_external_avg_rssi, scan_external_counts, scan_external_loud_bssid, scan_external_loud_ssid, scan_external_max_rssi, scan_insite_avg_rssi, scan_insite_counts, scan_insite_max_rssi, scan_noise_floor, scan_util_nonwifi, scan_util_rxotherbss, site, timestamp, tx_power, util_all, util_nonwifi, util_rxinbss, util_rxotherbss, util_tx, partition_0, dt
['5c5b352e113f', '5', '20', '157', '0.4598612', '11', '-87', '0', '56de201d-e63b-4312-9858-40f4cfe35c7f', '0', '153', '0.46477368', '0.002456248', '0.14768192', '-100', '0', '', '', '-100', '-54', '12', '-54', '-85', '0.0026979418', '0.15481576', 'a40ba08c-5e27-40d9-b798-7020ca6e4cae', '1970-01-18 20:09:18.158', '', '0.16867808', '0.0020744742', '2.6931155E-5', '0.1438904', '0.015231055', 'rrm-apscan-production', '2018-11-04']
['5c5b352e113f

In [23]:
import mistAPI
import mistGraph
userSiteID = "a40ba08c-5e27-40d9-b798-7020ca6e4cae"
userOrgID = ""
band = "5"

papi_url = 'http://papi-{ENV}.mistsys.net'.format(ENV=env)

In [24]:
rrmSiteConfig= mistAPI.getRRMBySite(userSiteID, band, papi_url)


{"command": "log", "msg": "getRRMBySite  http://papi-production.mistsys.net/internal/sites/a40ba08c-5e27-40d9-b798-7020ca6e4cae/devices/rrm/5 ", "level": 2}
end
{"command": "log", "msg": "query http://papi-production.mistsys.net/internal/sites/a40ba08c-5e27-40d9-b798-7020ca6e4cae/devices/rrm/5 None query-status=[200, 'OK', 0.818845] ", "level": 2}
end


In [25]:
mistG =  mistGraph.createGraphFromSiteConfig(rrmSiteConfig)

{"command": "log", "msg": "===== create-graph site=a40ba08c-5e27-40d9-b798-7020ca6e4cae, band=5, bandWidth=20,\n                    channels= [161, 157, 153, 149, 48, 44, 40, 36], site_power=0, dfs=True", "level": 2}
end
{"command": "log", "msg": " site=a40ba08c-5e27-40d9-b798-7020ca6e4cae nAPs = 133 channels[161, 157, 153, 149, 48, 44, 40, 36] ", "level": 2}
end


In [26]:
mistG.graph

{'org': '56de201d-e63b-4312-9858-40f4cfe35c7f',
 'site': 'a40ba08c-5e27-40d9-b798-7020ca6e4cae',
 'band': '5',
 'channels': [161, 157, 153, 149, 48, 44, 40, 36],
 'bandwidth': 20,
 'site_power': 0,
 'dfs': True,
 'ssid': [],
 'bssids': [],
 'aps': 133,
 'channels_chisq': 31.71763324987671,
 'naps_by_channel': {161: 11,
  153: 17,
  48: 14,
  157: 17,
  40: 14,
  36: 16,
  149: 14,
  44: 13,
  64: 5,
  60: 4,
  56: 4,
  52: 4},
 'avg_aps_per_channel': 16.625,
 'scanned_rssi': [-60, -50, -70],
 'when': '2018-11-13T15:23:53',
 'rrm_enabled': True,
 'reason': ''}

In [30]:
len(mistG)

133

In [31]:
mistGraph.getGraphPerformance(mistG)

{"command": "log", "msg": "WARN tot_entropy=1.1370947768858297 [161, 157, 153, 149, 48, 44, 40, 36] {161: 11, 153: 17, 48: 14, 157: 17, 40: 14, 36: 16, 149: 14, 44: 13, 64: 5, 60: 4, 56: 4, 52: 4}", "level": 3}
end
{"command": "log", "msg": " graph-updated=0 naps=133", "level": 2}
end


(31.71763324987671,
 {161: 11,
  153: 17,
  48: 14,
  157: 17,
  40: 14,
  36: 16,
  149: 14,
  44: 13,
  64: 5,
  60: 4,
  56: 4,
  52: 4})

In [32]:
mistG.graph

{'org': '56de201d-e63b-4312-9858-40f4cfe35c7f',
 'site': 'a40ba08c-5e27-40d9-b798-7020ca6e4cae',
 'band': '5',
 'channels': [161, 157, 153, 149, 48, 44, 40, 36],
 'bandwidth': 20,
 'site_power': 0,
 'dfs': True,
 'ssid': [],
 'bssids': [],
 'aps': 133,
 'channels_chisq': 31.71763324987671,
 'naps_by_channel': {161: 11,
  153: 17,
  48: 14,
  157: 17,
  40: 14,
  36: 16,
  149: 14,
  44: 13,
  64: 5,
  60: 4,
  56: 4,
  52: 4},
 'avg_aps_per_channel': 16.625,
 'scanned_rssi': [-60, -50, -70],
 'when': '2018-11-13T15:23:53',
 'rrm_enabled': True,
 'reason': '',
 'avgNoise': -90.0,
 'density': 0,
 'avgNeighbors': 0.0,
 'avgCochannelNeighbors': 0.0,
 'avgLoudNeighbors': 0.0,
 'avgLoudCochannelNeighbors': 0.0,
 'avgCochannelNeighbors_ref': 1.0,
 'aps_off': 0,
 'aps_channel_fixed': 20,
 'aps_power_fixed': 20,
 'naps_by_power': {11: 133},
 'channels_chisq_max': 0.004182156133828997,
 'channel_max': 153,
 'channels_entropy': 1.0,
 'isAnomaly': True,
 'apscores_mean': 0.40364068064899095,
 'aps